# Turning data into two dimensional tensor

In [11]:
require 'nn';

In [12]:
WordSPlitterMinibatchLoader = {}
WordSPlitterMinibatchLoader.__index = WordSPlitterMinibatchLoader

data_dir = "/Users/david/Documents/MemoryNetwork/output_lua"

function WordSPlitterMinibatchLoader.create_vocabulary(input_file,vocab_file)
    	print('loading text file....')
	local rawdata
	local tot_len = 0
	local f = assert(io.open(input_file,"r"))
	local max_sent_len = 0
	local sent_count = 0
	-- Create vocabulary if it doesn't exist yet
	print('creating vocabulary mapping')
	local unordered = {}
	rawdata = f:read():lower()
	repeat
		sent_count = sent_count + 1
		for k,word in pairs(rawdata:split(" ")) do 
			word=word:lower()
			if not unordered[word] then unordered[word] = true end
		end
		sent_len = #rawdata:split(" ")
		if sent_len > max_sent_len then max_sent_len=sent_len end
		tot_len = tot_len + sent_len
		rawdata = f:read()
	until not rawdata
	f:close()
	-- sort into a table (i.e. keys become 1..N)
	local ordered = {}
	for word in pairs(unordered) do ordered[#ordered + 1] = word end
	table.sort(ordered)
	-- invert `ordered` to create the char->int mapping
	local vocab_mapping = {}
	for i, word in ipairs(ordered) do
		vocab_mapping[word] = i
	end
	print('saving ' .. vocab_file)
    torch.save(vocab_file, vocab_mapping)
    return {sent_count,vocab_mapping,max_sent_len}
end

In [13]:
function WordSPlitterMinibatchLoader.create_tensor(sent_count,vocab_mapping,max_sent_len,input_file,tensor_file)
	print('putting data into tensor...')
	local data = torch.ByteTensor(sent_count,max_sent_len):zero() -- store it into 1D first, then rearrange
	f = assert(io.open(input_file,"r"))
	local currline = 1
	-------- Writing in the tensor file 
	rawdata = f:read()
	repeat
        rawdata = rawdata:lower()
		for k,word in pairs(rawdata:split(" ")) do 
			data[{currline,k}] = vocab_mapping[word:lower()]
		end
		currline = currline + 1
		rawdata = f:read()
	until not rawdata
	f:close()
	-- save output preprocessed files
    print('saving ' .. tensor_file)
    torch.save(tensor_file, data)
end


In [14]:
function WordSPlitterMinibatchLoader.text_to_tensor(input_file, out_vocab_file, out_tensor_file)
    local timer = torch.Timer()
    res = WordSPlitterMinibatchLoader.create_vocabulary(input_file,out_vocab_file)
    local sent_count = res[1]
    local vocab_mapping = res[2]
    local max_sent_len = res[3]
    return WordSPlitterMinibatchLoader.create_tensor(sent_count,vocab_mapping,max_sent_len,input_file,out_tensor_file)
end

In [15]:
input_file = "/Users/david/Documents/MemoryNetwork/preprocessing/output.txt"
out_vocab_file = "/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7"
out_tensor_file = "/Users/david/Documents/MemoryNetwork/output_lua/data.t7"

WordSPlitterMinibatchLoader.text_to_tensor(input_file,out_vocab_file,out_tensor_file)

loading text file....	
creating vocabulary mapping	


saving /Users/david/Documents/MemoryNetwork/output_lua/vocab.t7	


putting data into tensor...	


saving /Users/david/Documents/MemoryNetwork/output_lua/data.t7	


# Exploring data + building model

In [16]:
data = torch.load(out_tensor_file)
vocab = torch.load(out_vocab_file)

In [17]:
data:size()

 70
 57
[torch.LongStorage of size 2]



## Easy model - with nngraph

In [18]:
require 'nngraph';
require 'math';

In [3]:
require 'nn';

In [ ]:
debug.getregistry()["nn.LSTM"] = nil
layer, parent = torch.class('nn.LSTM', 'nn.Module')

In [80]:
L = 5
rnn_size = 10
input_size = data:size(2) 

In [81]:
for i=1,L do 
    inputs = {}
    table.insert(inputs,nn.Identity()())
    table.insert(inputs,nn.Identity()())
end

In [82]:
for  i = 1, L do
    if i == 1 then
        input_size_L = input_size
    else
        input_size_L = rnn_size
    end
end

In [83]:
e = torch.Tensor(10):normal(0)
a = {}
o = {}
table.insert(a,nn.Identity()())
r = nn.Reshape(2,5)(a[1])
--s = nn.SplitTable(2)(r)
table.insert(o,r)

g = nn.gModule(a, o)
graph.dot(g.fg, 'MLP', 'myMLP')

In [39]:
g:forward(e)

-0.0291  0.1834 -1.5732 -0.1059 -0.2625
-0.3480  0.5151 -0.0530  0.5272 -4.0310
[torch.DoubleTensor of size 2x5]



## Random Experiments

In [168]:
--
inp = torch.Tensor(10,20,40):normal(0)
dim_rnn = 100

In [192]:
-- generating the required
inputs = {}
table.insert(inputs,nn.Identity()())
i2h = nn.Linear(inp:size(3),4*dim_rnn)(inputs):annotate{name='Transition input -> hidden'}
h2h = nn.Linear(dim_rnn,dim_rnn)

In [218]:
inputs = {}
outputs = {}
prev_h = torch.Tensor(5)
table.insert(inputs,nn.Identity()())
table.insert(inputs,nn.Identity()())
i2h = nn.Linear(5,50)(inputs[1]):annotate{comment="This is a very cool annotation"}
h2h = nn.Linear(50,50)(inputs[2]):annotate{comment="And this one is a pretty bad one"}
sum = nn.CAddTable()({i2h,h2h})
table.insert(outputs,sum)
nimp = nn.gModule(inputs, outputs)

In [214]:
a = {torch.Tensor(5):normal(0),torch.Tensor(50):normal(0)}
aa = nimp:forward(a)

## Building model

In [48]:
require "nn";
require 'math';

For N = 100, D = 512, T = 100, H = 1024 and with 4 bytes per number, this comes
out to 305MB. Note that this class doesn't own input or gradOutput, so you'll
see a bit higher memory usage in practice.

In [103]:
debug.getregistry()["nn.LSTM"] = nil
layer, parent = torch.class('nn.LSTM', 'nn.Module')

function layer:__init(input_dim,hidden_dim)
    parent.__init(self) 
    local D,H = input_dim, hidden_dim
    self.input_dim, self.hidden_dim = D, H
    self.weight = torch.Tensor(D+H,4*4)
    self.gradWeight = torch.Tensor(D+H,4*H):zero()
    self.bias = torch.Tensor(4*H)
    self.gradBias = torch.Tensor(4*H):zero()
    self.reset()
    
    self.cell = torch.Tensor()
    self.gates = torch.Tensor()
    self.buffer1 = torch.Tensor()
    self.buffer2 = torch.Tensor()
    self.buffer3 = torch.Tensor()
    self.grad_a_buffer = torch.Tensor()
    
    self.h0 = torch.Tensor()
    self.c0 = torch.Tensor()
    self.remember_states = false
    
    self.grad_c0 = torch.Tensor()
    self.grad_h0 = torch.Tensor()
    self.grad_x = torch.Tensor()
    
    self.gradInput = {self.grad_c0, self.grad_h0, self.grad_x}
end

function layer:reset(std)
    if not std then
        std = 1.0 / math.sqrt(self.hidden_dim + self.input_dim)
    end
    self.bias:zero()
    self.bias[{{self.hidden_dim + 1, 2*self.hidden_dim}}]:fill(1)
    self.weight:normal(0,std)
    return self
end


function layer:_unpack_input(input)
    local c0, h0, x = nil, nil, nil 
    if torch.type(input) == "table" and #input == 3 then
        c0,h0,x = unpack(input)
    elseif torch.type(input) == "table" and #input == 2 then
        h0,x = unpack(input)
    elseif torch.isTensor(input) then
        x = input
    else 
        assert(false,'invalid input')
    end
    return c0,h0,x
end


function layer:_get_sizes(input, gradOutput)
  local c0, h0, x = self:_unpack_input(input)
  local N, T = x:size(1), x:size(2)
  local H, D = self.hidden_dim, self.input_dim
  check_dims(x, {N, T, D})
  if h0 then
    check_dims(h0, {N, H})
  end
  if c0 then
    check_dims(c0, {N, H})
  end
  if gradOutput then
    check_dims(gradOutput, {N, T, H})
  end
  return N, T, D, H
end

function layer:updateOutput(input)
    self.recompute_backward = true 
    local c0, h0, x = self:_unpack_input(input)
    local N, T, D, H = self:_get_sizes(input)
    
    self._return_grad_c0 = (c0~=nil)
    self._return_grad_h0 = (h0~=nil)
    
    -- initialize the Cell component
    if not c0 then
        c0 = self.c0
        if c0:nElement() == 0 or not self.remember_states then
            c0:resize(N,H):zero()
        elseif self.remember_states then
            local prev_N, prev_T = self.cell:size(1), self.cell:size(2)
            assert(prev_N==N,'barch sizes must be contant to remember states')
            c0:copy(self.cell[{{},prev_T}])
        end
    end
    
    -- initialize the Hidden States component    
    if not h0 then
        h0 = self.h0
        if h0:nElement() == 0 or not self.remember_states then
            h0:resize(N,H):zero()
        elseif self.rember_states then
            local prev_N, prev_T = self.output:size(1), self.output:size(2)
            assert(prev_N == N, "batch sizes must be the same to remember states")
            h0:copy(self.output[{{},prev_T}])
        end
    end
    
    local bias_expand = self.bias:view(1,4*H):expand(N,4*H)
    local Wx = self.weight[{{1,D}}]
    local Wh = self.weight[{{D+1,D+H}}]
    
    local h,c = self.output, self.cell
    h:resize(N,T,H):zero()
    c:resize(N,T,H):zero()
    local prev_h, prev_c = h0, c0
    self.gates:resize(N,T,4*H)
    for t=1,T do 
        local cur_x = x[{{},t}]
        local next_g = h[{{},t}]
        local next_c = c[{{},t}]
        local cur_gates = self.gates[{{},t}]
        cur_gates:addmm(bias_expand,cur_x,Wx)
        cur_gates:addmm(prev_h,Wh)
        cur_gates[{{}, {1, 3 * H}}]:sigmoid()
        cur_gates[{{}, {3 * H + 1, 4 * H}}]:tanh()
        local i = cur_gates[{{}, {1, H}}]
        local f = cur_gates[{{}, {H + 1, 2 * H}}]
        local o = cur_gates[{{}, {2 * H + 1, 3 * H}}]
        local g = cur_gates[{{}, {3 * H + 1, 4 * H}}]
        next_h:cmul(i,g)
        next_c:cmul(f,prev_c):add(next_h)
        next_h:tanh(next_c):cmul(o)
        prev_h, prev_c = next_h,next_c
    end
    return self.output
end

function layer:backward(input, gradOutput, scale)
    self.recompute_backward= false
    scale = scale or 1.0
    assert(scale == 1.0, 'must have scale=1')
    local c0, h0, x = self._unpack_input(input)
    if not c0 then c0 = self.c0 end
    if not h0 then h0 = self.h0 end
    
    local grad_c0, grad_h0, grad_x = self.grad_c0, self.grad_h0, self.grad_x
    local h, c = self.output, self.cells
    local grad_h = gradOutput
    
    local N, T, D, H = self:_get_sizes(input,gradOutput)
    local Wx = self.weight[{{1,D}}]
    local Wh = self.weight[{{D+1,D+H}]
        
    local grad_Wx = self.gradWeight[{{1,D}}]
    local grad_Wh = self.gradWeight[{{D+1,D+H}}]
    local grad_b = self.gradBias
        
    grad_h0:resizeAs(h0):zero()
    grad_c0:resizeAs(c0):zero()
    for t = T, 1, -1 do
        local next_h, next_c = h[{{},t}], c[{{},t}]
        local tanh_next_c2 = grad_af:cmul(tanh)
end

[string "debug.getregistry()["nn.LSTM"] = nil..."]:81: ')' expected near '=': 

In [94]:
c0 = torch.Tensor(10,3)

In [110]:
c = torch.Tensor(5,3,10)
print(c:size())

  5
  3
 10
[torch.LongStorage of size 3]



In [ ]:
c:size()

In [119]:
a = torch.Tensor()
c:view(a,10,5,-1)

/Users/david/torch/install/share/lua/5.1/torch/Tensor.lua:462: bad argument #1 to 'set' (expecting number or torch.DoubleTensor or torch.DoubleStorage at /Users/david/torch/pkg/torch/generic/Tensor.c:1153)
stack traceback:
	[C]: in function 'set'
	/Users/david/torch/install/share/lua/5.1/torch/Tensor.lua:462: in function 'view'
	[string "a = torch.Tensor()..."]:2: in main chunk
	[C]: in function 'xpcall'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:210: in function </Users/david/torch/install/share/lua/5.1/itorch/main.lua:174>
	/Users/david/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(comman

In [ ]:
a = nn.LSTM(10,100)
a.weight:size()

In [43]:
require 'nngraph';
GRU = {}

function GRU.instanciate_model(input_size,rnn_size,n)
    print(a)
end

 110
  16
[torch.LongStorage of size 2]



# Scrap

In [99]:
aaaa = torch.Tensor(10)
print(aaaa)
aaaa = aaaa:div(4)
print(aaaa)

 -0.0000e+00
 -0.0000e+00
  4.9046e-62
  4.7607e-90
 4.7478e+174
 3.9706e+246
  1.1632e-28
 8.7644e+169
  2.3278e-57
  1.4270e-71
[torch.DoubleTensor of size 10]



 -0.0000e+00
 -0.0000e+00
  1.2262e-62
  1.1902e-90
 1.1869e+174
 9.9266e+245
  2.9080e-29
 2.1911e+169
  5.8196e-58
  3.5674e-72
[torch.DoubleTensor of size 10]

